<a href="https://colab.research.google.com/github/sh1027/detectron2_tutorial/blob/main/detectron2_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detectron2 チュートリアル
<img src="https://dl.fbaipublicfiles.com/detectron2/Detectron2-Logo-Horz.png" width="500">

本チュートリアルでは，Detectron2の事前学習済みモデルを用いて，物体検出やその関連タスクの推論を試します．

✅ 推論を試すタスク

*   物体検出 (Object Detection)
*   Instance Segmentation
*   Keypoint Detection
*   Panoptic Segmentation

✅ 推論を試す入力

*   COCO Datasetの画像
*   Google Drive内の画像
*   Google Drive内の動画
*   Youtubeの動画

# 目次

1.   セットアップ
2.   COCOデータセットの画像を用いた推論（物体検出）
3.   COCOデータセットの画像を用いた推論（関連タスク）
4.   Google Drive内の画像を用いた推論
5.   動画を用いた推論







# 1. セットアップ

pytorchと，pytorchのバージョンに合ったdetectron2をインストールします．

実行後にランタイムを再起動してください．

In [ ]:
!pip install torch==1.10.0+cu111 torchvision==0.11.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install detectron2 -f \https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html

ランタイムの再起動後に，pytorchのバージョンをチェックします．以下のように出力されればOKです．

```
torch version           : 1.10.0+cu111
torch cuda is available : True
detectron2 version      : 0.6
```

In [ ]:
import torch
import detectron2
print("torch version           :", torch.__version__)
print("torch cuda is available :", torch.cuda.is_available())
print("detectron2 version      :", detectron2.__version__)

In [ ]:
# import some common libraries
import numpy as np
import os, cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# 2. COCOデータセットの画像を用いた推論（物体検出）



## COCOデータセットの読み込み
COCOデータセットから画像をダウンロードします．
まずは，画像を表示してみましょう．

In [ ]:
!wget http://images.cocodataset.org/val2017/000000439715.jpg -q -O coco_input.jpg
img = cv2.imread("./coco_input.jpg")
cv2_imshow(img)

## 推論用のconfigを設定
Detectron2にはデフォルトの設定があるので，変更が必要なところだけ変更をします．

ここで事前学習済みファイルを読み込むことで，事前学習済みモデルを使用することができます．

In [ ]:
# Detectron2のデフォルトの設定ファイルをコピー
cfg = get_cfg()
# Faster R-CNNの事前学習済みモデル(のチェックポイント)と，モデル固有の設定ファイルを読み込み
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
# 出力するbounding boxのスコアの閾値を設定
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7

## 推論の実行
Detectron2には推論のためのシンプルなクラス (`DefaultPredictor`) が用意されています．それを使用することで簡単に推論が可能です．

In [ ]:
predictor = DefaultPredictor(cfg)
outputs = predictor(img)

## 推論結果の確認
推論結果が格納されている`outputs`の中身を確認していきましょう．
以下のリンク先のドキュメントには，`outputs`の形式が示されています．細かい形式を確認したい人は[公式ドキュメント](https://detectron2.readthedocs.io/en/latest/tutorials/models.html#model-output-format)を確認してみてください．

**outputsの型とkeyを確認**

`outputs`は辞書型となっています．今回のFaster R-CNNの推論結果では`instances`のkeyのみ入ってることが確認できます．

In [ ]:
print(type(outputs))
print(outputs.keys())

**インスタンスの中身を確認**

`outputs["instances"]`の中身を確認してみます．
Instancesというクラスになっており，fieldsのところに

*   pred_boxes: 推論したbounding box
*   scores: 推論したbounding boxのスコア
*   pred_classes: 推論したbounding boxのクラス

が入っていることが確認できます．
(参考: 
 [Instancesクラスのドキュメント](https://detectron2.readthedocs.io/en/latest/modules/structures.html#detectron2.structures.Instances))


In [ ]:
instances = outputs["instances"]
instances

## 推論結果の可視化
Detectron2では，`Visualizer`クラスを用いて推論結果の可視化を簡単に行うことができます．

In [ ]:
metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
vis = Visualizer(img[:,:,::-1], metadata)
out = vis.draw_instance_predictions(instances.to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

**メタデータの確認**

上記コードに登場する`metadata`には，クラスラベルや色など，データセットのメタデータが入っています．このメタデータを用いて，`Visualizer`クラスのメソッドは推論結果の可視化を行っています．



In [ ]:
# 今回の可視化に用いるメタデータを確認
metadata

また，メタデータの`thing_classes`を見ると，先ほどの`outputs`の`pred_classes`のラベルと名前を対応させることができます．例えば，`pred_classes`が0のところは，personだったことが分かります．

In [ ]:
# ラベルが0のクラス名を確認
metadata.get("thing_classes")[0]

## プチ演習: 推論結果のうち，馬のみを可視化してみよう
 [Instancesクラスのドキュメント](https://detectron2.readthedocs.io/en/latest/modules/structures.html#detectron2.structures.Instances)に書いてある通り，`Instances`クラスは条件を満たす部分インスタンスを抽出することができます．以下の例を参考にして，馬のインスタンスのみを抽出してみましょう．


```
例: 
category_3_detections = instances[instances.pred_classes == 3]
confident_detections = instances[instances.scores > 0.9]
```

以下のコードを変更して，馬の検出結果のみを可視化してみましょう

In [ ]:
###  ここから変更  ###
horse_detections = instances
### ここまでを変更 ###

vis = Visualizer(img[:,:,::-1],MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
out = vis.draw_instance_predictions(horse_detections.to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

**正解 (❗答え合わせのときに表示してください❗)**


In [ ]:
#@title
###  ここから変更  ###
horse_detections = instances[instances.pred_classes == 17]
### ここまでを変更 ###

vis = Visualizer(img[:,:,::-1],MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
out = vis.draw_instance_predictions(horse_detections.to("cpu"))
cv2_imshow(out.get_image()[:, :, ::-1])

# 3. COCOデータセットの画像を用いた推論（関連タスク）

Detectron2を用いて，物体検出 (Object Detection)の推論を試すことができたので，次は
*   Instance Segmentation
*   Keypoint Detection
*   Panoptic Segmentation

の推論を試します．

## Detectorクラスの作成

物体検出を試して分かる通り，全てのタスクで，事前学習済みモデルを読み込むところ以外はほとんど同じ操作で推論できます．そのため，今後はDetectorクラスに操作をまとめて実行していきます．

In [ ]:
class Detector:
  def __init__(self, model_type, th=0.7):
    self.cfg = get_cfg()
    self.model_type = model_type

    if model_type == "OD": # object detection
      model_path = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"
    elif model_type == "IS": # instance segmentation
      model_path = "COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
    elif model_type == "KP": # keypoint detection
      model_path = "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
    elif model_type == "PS": # panoptic segmentation
      model_path = "COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"
    else:
      raise NotImplementedError()
    
    # load model config and pretrained model
    self.cfg.merge_from_file(model_zoo.get_config_file(model_path))
    self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_path)
    # set threshold for this model
    self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = th  
    # cpu or cuda
    self.cfg.MODEL.DEVICE = "cuda" 
    
    self.predictor = DefaultPredictor(self.cfg)
  
  def _draw_predictions(self, img):
    viz = Visualizer(img[:,:,::-1], metadata=MetadataCatalog.get(self.cfg.DATASETS.TRAIN[0]))
    if self.model_type == "PS":
      predictions, segments_info = self.predictor(img)["panoptic_seg"]
      out = viz.draw_panoptic_seg_predictions(predictions.to("cpu"), segments_info)
    else:
      predictions = self.predictor(img)
      out = viz.draw_instance_predictions(predictions["instances"].to("cpu"))
    return out

  def on_image(self, src_path, dst_width=640):
    img = cv2.imread(src_path)

    # resize if img is too large
    h, w = img.shape[:2]
    if w > dst_width:
      dst_height = round(h * (dst_width / w))
      img = cv2.resize(img, dsize=(dst_width, dst_height))
    
    out = self._draw_predictions(img)    
    cv2_imshow(out.get_image()[:, :, ::-1])
  
  def on_video(self, src_path, dst_path='./video_out.mp4', dst_width=640):
    cap = cv2.VideoCapture(src_path)
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = float(cap.get(cv2.CAP_PROP_FPS))

    if w > dst_width:
      dst_height = round(h * (dst_width / w))
    else:
      dst_height = h
    fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
    rec = cv2.VideoWriter(dst_path, fourcc, fps, (dst_width, dst_height))

    if not cap.isOpened():
      print("Error opening the video...")
      return
    
    while True:
      ret, frame = cap.read()
      if not ret:
        break
      
      resized_frame = cv2.resize(frame, dsize=(dst_width, dst_height))
      out = self._draw_predictions(resized_frame)

      rec.write(out.get_image()[:, :, ::-1])
    
    rec.release()

## Object Detection

先ほど試した物体検出の推論は，Detectorクラスを用いて以下のように書くことができるようになりました．

In [ ]:
coco_img_path = "./coco_input.jpg"
detector = Detector("OD")
detector.on_image(coco_img_path)

## Instance Segmentation

In [ ]:
detector = Detector("IS")
detector.on_image(coco_img_path)

## Keypoint Detection

In [ ]:
detector = Detector("KP")
detector.on_image(coco_img_path)

## Panoptic Segmentation

In [ ]:
detector = Detector("PS")
detector.on_image(coco_img_path)

# 4. Google Drive内の画像を用いた推論

各自用意してもらった画像で，物体検出や関連タスクの推論をしてもらいます！

Google Driveのマイドライブ内に，推論を試してみたい画像を保存してください．
（マイドライブ直下だと操作が簡単です．）

## ドライブをマウント

Google Drive内のファイルにアクセスするには，マウントが必要です．

以下のセルを実行後，ポップアップが表示されます．「Googleドライブに接続」をクリックして，ドライブを接続するアカウントを選択し，接続を許可してください．

`Mounted at /content/drive`と表示されれば成功です.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 推論に用いる画像の準備

`img_path_from_drive`に，マイドライブからの画像のパスを代入してください．
マイドライブ直下の場合は，画像のファイル名でOKです．

In [ ]:
# マイドライブからの画像のパスを代入
img_path_from_drive = "drive_input.jpg"

In [ ]:
# このセルは変更しない
drive_path = "drive/MyDrive"
drive_img_path = os.path.join(drive_path, img_path_from_drive)

**推論に用いる画像を表示**

ドライブのマウントと画像のパスの設定が正しくできていれば，推論に用いる画像が表示されます．

In [ ]:
img = cv2.imread(drive_img_path)

# resize if img is too large
dst_width = 640
h, w = img.shape[:2]
if w > dst_width:
  dst_height = round(h * (dst_width / w))
  img = cv2.resize(img, dsize=(dst_width, dst_height))

cv2_imshow(img)

## 推論の実行

In [ ]:
# Object Detection
detector = Detector("OD")
detector.on_image(drive_img_path)

In [ ]:
# Instance Segmentation
detector = Detector("IS")
detector.on_image(drive_img_path)

In [ ]:
# Keypoint Detection
detector = Detector("KP")
detector.on_image(drive_img_path)

In [ ]:
# Panoptic Segmentation
detector = Detector("PS")
detector.on_image(drive_img_path)

# 5. 動画を用いた推論

## ドライブ内の動画を用いた推論

### セットアップ

In [ ]:
from IPython.display import HTML
import base64
import io

# Google Colabolatoryで動画を再生するための関数
def play(file_path):
    video = io.open(file_path, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))

### 推論に用いる動画の準備

`video_path_from_drive`に，マイドライブからの動画のパスを代入してください．
マイドライブ直下の場合は，動画のファイル名でOKです．

**❗注意❗**
動画が長すぎると処理が重たくなってしまうため，動画の長さは**10秒以内**程度に収めてください．

In [ ]:
# ドライブをマウント（マウントができていない場合）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# マイドライブからの画像のパスを代入
video_path_from_drive = "drive_input.mp4"

In [ ]:
# このセルは変更しない
drive_path = "drive/MyDrive"
drive_video_path = os.path.join(drive_path, video_path_from_drive)

### 推論の実行
以下の例では，Panoptic Segmentationの例を載せています．
`Detector`の引数を変えて，試してみたいタスクを実行してみてください．

In [ ]:
# Panoptic Segmentation
detector = Detector("PS")

In [ ]:
# このセルは変更しない
dst_path = "./drive_out.mp4"
play_path = "./out.webm"

detector.on_video(src_path=drive_video_path, dst_path=dst_path)
!ffmpeg -i '{dst_path}' -vcodec vp9 '{play_path}'
play(play_path)

## Youtubeの動画を用いた推論

### セットアップ

In [ ]:
# Youtube動画をダウンロードするライブラリのインストール
!pip install yt-dlp

In [ ]:
from IPython.display import YouTubeVideo, display, HTML
import base64
import io

# Google Colabolatoryで動画を再生するための関数
def play(file_path):
    video = io.open(file_path, 'r+b').read()
    encoded = base64.b64encode(video)
    return(HTML(data='''<video controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii'))))

### 推論に用いる動画の準備

推論に用いる**動画のIDを取得してください**．動画のIDはURLから分かります．


---

例1.   https://www.youtube.com/watch?v=ll8TgCZ0plk のようなアドレスの場合

  → 動画IDは`?v=`の後の`ll8TgCZ0plk`です．もし`&`が出てくる場合は，`&`の手前までです．

例2.   https://youtu.be/ll8TgCZ0plk のようなアドレスの場合

  → 動画IDは`youtu.be/`の後の`ll8TgCZ0plk`です．

---

Youtubeを開いて動画下部の「共有」を押して表示されるURLは例2のパターンとなるので，分からない場合は**「共有」からリンクを取得するのが分かりやすいです**．


In [ ]:
# 推論に用いる動画のIDを代入
youtube_video_id = "ll8TgCZ0plk"

**推論に用いる動画を表示**

YoutubeのIDの設定が正しくできていれば，推論に用いる動画が表示されます．

In [ ]:
video = YouTubeVideo(youtube_video_id, width=640)
display(video)

**動画のダウンロード**

In [ ]:
youtube_video_link = f"https://www.youtube.com/watch?v={youtube_video_id}"
!yt-dlp '{youtube_video_link}' -f 22 -o youtube_input.mp4

**動画の切り出し**

Youtubeの動画をそのまま使うと長くなってしまうので，短い時間に切り出します．下のセルでは，最初から10秒間を切り出します．

もし切り出しの開始時間を指定したい場合は，下記の例のように，`-ss`オプションを追加してください．ただし，`-ss`の値通りに切り出されない (不正確な) 可能性があります． (参考: [FFmpegのwiki](https://trac.ffmpeg.org/wiki/Seeking))


```
# 5分地点から10秒間の動画を切り出す例
!ffmpeg -ss 300 -i youtube_input.mp4 -t 10 -c:v copy youtube_input_clip.mp4
```



In [ ]:
!ffmpeg -i youtube_input.mp4 -t 10 -c:v copy youtube_input_clip.mp4

### 推論の実行
以下の例では，Panoptic Segmentationの例を載せています．
`Detector`の引数を変えて，試してみたいタスクを実行してみてください．

In [ ]:
# Panoptic Segmentation
detector = Detector("PS")

In [ ]:
# このセルは変更しない
src_path = "./youtube_input_clip.mp4"
dst_path = "./youtube_out.mp4"
play_path = "./out.webm"

detector.on_video(src_path=src_path, dst_path=dst_path)
!ffmpeg -i '{dst_path}' -vcodec vp9 '{play_path}'
play(play_path)